In [ ]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.3 MB/s eta 0:00:00


In [ ]:
import cv2
import os
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_folder = "/content/drive/MyDrive/Project/test_violence_dataset"


In [ ]:
# Load YOLOv8 nano model (fastest)
model = YOLO("yolov8n.pt")

In [ ]:
# Output folder (results will be saved here)
output_folder = "/content/drive/MyDrive/Project/blurred3_output"
os.makedirs(output_folder, exist_ok=True)


In [ ]:
import numpy as np

for i, img_name in enumerate(image_files, start=1):
    img_path = os.path.join(input_folder, img_name)

    print(f"Processing {i}/{len(image_files)}: {img_name}")

    frame = cv2.imread(img_path)
    if frame is None:
        continue

    # ===== Option 3: Anime weapon masking =====
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Adjust these ranges to match anime swords/guns
    lower_gray = np.array([0, 0, 50])
    upper_gray = np.array([180, 50, 200])
    mask = cv2.inRange(hsv, lower_gray, upper_gray)

    # Optional: refine mask
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.dilate(mask, kernel, iterations=1)

    # Apply blur to masked regions
    blurred = cv2.GaussianBlur(frame, (25, 25), 15)
    frame = np.where(mask[:, :, None] == 255, blurred, frame)

    # ===== (Optional) Keep YOLO detection for other harmful objects like "person" =====
    results = model(frame, verbose=False)
    for r in results:
        for box, cls_id in zip(r.boxes.xyxy, r.boxes.cls):
            class_name = model.names[int(cls_id)]
            if class_name in ["person", "umbrella"]:  # remove weapons here
                frame = blur_region(frame, box)

    # Save output image
    save_path = os.path.join(output_folder, img_name)
    cv2.imwrite(save_path, frame)

print("All images processed! Blurred results saved in:", output_folder)



Processing 1/68: video5-mp4_segment_7-mp4_795_jpg.rf.158f8cadb04639a00451025e99952683.jpg
Processing 2/68: video5-mp4_segment_10-mp4_1050_jpg.rf.2aa5547bd35ed725256a4eef75333bf2.jpg
Processing 3/68: video39-mp4_segment_10-mp4_45_jpg.rf.c9a6d493ed31d7f75bd6e2ff780a19c5.jpg
Processing 4/68: video7-mp4_segment_8-mp4_245_jpg.rf.0d042541a1543b42c0f922e9b9355d52.jpg
Processing 5/68: video5-mp4_segment_10-mp4_30_jpg.rf.49ce2c4856bd904acbba0d6b50fdd153.jpg
Processing 6/68: video6-mp4_segment_4-mp4_7685_jpg.rf.b1927474f5ae6ea8db49dc5fe9843f67.jpg
Processing 7/68: video6-mp4_segment_7-mp4_2435_jpg.rf.28b564806adb8b21e0fa98fa0c159655.jpg
Processing 8/68: video8-mp4_segment_7-mp4_25_jpg.rf.8c61b3609b6f9cee69bd4d0542a21d9e.jpg
Processing 9/68: video6-mp4_segment_7-mp4_1260_jpg.rf.d66ca72f371929a69be96f996cf38a81.jpg
Processing 10/68: video6-mp4_segment_4-mp4_12310_jpg.rf.b14b589811896f805979d6d46ec4dff9.jpg
Processing 11/68: video5-mp4_segment_10-mp4_870-Copie_jpg.rf.52240e8f0a6c4d6a681e2301a669fda

In [ ]:
for r in results:
    for box, cls_id in zip(r.boxes.xyxy, r.boxes.cls):
        class_name = model.names[int(cls_id)]
        print("Detected:", class_name)


Detected: person
Detected: umbrella
Detected: person
